In [ ]:
#Веб-сервис: скачивает стену сообщества, строит граф (узлы -- пользователи, связи -- факт диалога между ними), 
#визуализирует на странице. Размер узлов зависит от объёма сказанного
#скачать комменты к постам 8 274
#сделать словарь пользователь (id) - кол-во слов в сообщении
#сделать словарь пользователь (id) - кому отвечал (id)
#сделать словарь id - имя (first_name last_name)
#построить граф, сохранить картинку
#написать две странички на фласке: типа ГО и РЕЗ-ТЫ
#отправить на хероку

import urllib.request  # импортируем модуль
import ssl
import json
import matplotlib.pyplot as plt
import re
import networkx as nx
from flask import Flask
from flask import url_for, render_template, request, redirect
import os

app = Flask(__name__)

ssl._create_default_https_context = ssl._create_unverified_context

def get_posts(token):
    #offsets = [0, 150, 350]
    user_len={} #словарь пользователь (id) - кол-во слов в сообщении
    user_connection=[] #словарь пользователь (id) - кому отвечал (id)
    user_names={} #словарь id - имя (first_name last_name)
    len_comm=[]
    for off in range(1, 8274, 100): #не забыть исправить на 8к
        req = urllib.request.Request('https://api.vk.com/method/wall.get?owner_id=-55284725&count=100&v=5.74&access_token='+token+'&offset=' + str(off))
        response = urllib.request.urlopen(req)
        result = response.read().decode('utf-8')
        data = json.loads(result)
        for n in range(10):
            id_post = data["response"]["items"][n]["id"]
            num_comm=data["response"]["items"][n]["comments"]['count']
            for off_comm in range(1, num_comm, 100):
                stroka = 'https://api.vk.com/method/wall.getComments?owner_id=-55284725&post_id='+str(id_post)+'&count=100&v=5.74&access_token='+token+'&extended=1&fields=bdate,city&offset='+str(off_comm)              
                req1= urllib.request.Request(stroka)
                response1 = urllib.request.urlopen(req1)
                result1 = response1.read().decode('utf-8')
                data2 = json.loads(result1)
                    
                for m in range(len(data2['response']['profiles'])):
                    try:
                        profile=str(data2['response']['profiles'][m]['id'])   
                    except:
                        pass
                    info=data2['response']['items'][m]['text']
                    if info!='':
                        len_sum=len(info.split()) #длина коммента
                        if profile in user_len:
                            user_len[profile]+=len_sum 
                        else: 
                            user_len[profile]=len_sum
                            #словарь вершин
                        id_chat=connections(info)
                        if id_chat!=0:
                            user_connection.append((profile, id_chat)) #ребра графа
                            
                            
    return user_len, user_connection
                            

def connections(info):
    id_search='\[id[0-9]+'
    result = re.findall(id_search, info)
    try:
        result=result[0].replace('[id', '')
        return result
    except:
        return 0
    
def visualizing(user_len, user_connection):
    nodes=[]
    size=[]
    keys=dict.keys(user_len)
    for n in keys:
        nodes.append(n)
    values=dict.values(user_len)
    for v in values:
        size.append(v)
    max_size = sorted(size, reverse=True)[:5]
    labels=labels_graph(max_size, user_len)
    
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(user_connection)              
    nx.write_gexf(G, 'graph_file.gexf')
    pos=nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, node_color='blue', node_size=size)
    nx.draw_networkx_edges(G, pos, edge_color='grey')
    nx.draw_networkx_labels(G, pos, labels, font_size=7, font_family='Arial')
    plt.axis('off')
   
    plt.savefig('static/graph', dpi=200, format='png')
    return G

def labels_graph(max_size, user_len):
    labels={}
    for m in max_size:
        key=list(user_len.keys())[list(user_len.values()).index(m)]
        labels[key]=key
    return labels

@app.route('/')  #главная
def index():
    return render_template('index.html')

@app.route('/resp')
def resp():
    #path_image = os.path.abspath("graph.png")
    #path_image = '<img src="'+path_image+'">'
    #print(path_image)
    return render_template('thanks.html')
    

if not os.path.exists("static"):
    os.mkdir("static")                 
token="2cadf5de2cadf5de2cadf5de812cc93a4b22cad2cadf5de7789e7968ba4b50b222a95ae"
u, c = get_posts(token)
visualizing(u,c)

if __name__ == '__main__':
    app.run(debug=False)

c=connections(info)
